In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import gc
import os
from os import listdir
from os.path import isfile, join

import json
import zipfile
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import cv2

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:
# Extracting all of the zip files

# Training dataset
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as z:
     z.extractall('../data')
    
# Testing dataset
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as z:
    z.extractall('../data')


In [ ]:
os.listdir('../data')


In [ ]:
# Cat = 0, Dog = 1
def create_df():
    path = '../data/train'
    dataset = []
    for f in listdir(path):
        if isfile(join(path, f)):
            if 'cat' in f:
                dataset.append({'label': 0, 'img_path': join(path, f)})
            elif 'dog' in f:
                dataset.append({'label': 1, 'img_path': join(path, f)})

    df = pd.DataFrame(dataset)

    # shuffling the dataframe and resetting the index
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df


In [ ]:
# Getting the dataset
df = create_df()
df.sample(5)


In [ ]:
# converting labels dtype to str
# to use class_mode='binary' in ImageDataGenerators
df.label = df.label.astype('str')
df.sample(5)


In [ ]:
print(f'Dataset size: {len(df)}')


In [ ]:
# See if dataset is balanced or not

plt.figure(figsize=(8, 4))
plt.title('Number of classes')
g = sns.countplot(df.label, palette='icefire')
g.set(xlabel='Classes', ylabel='Count')
sns.despine(offset=5)
plt.show()


In [ ]:
# Splitting the training and validation sets
x_train, x_val, y_train, y_val = train_test_split(df.img_path, df.label, test_size=0.1, random_state=0)

# Splitting the remaining training set into training and test sets
# This set is different from the set ==> /kaggle/working/dogs-vs-cats/test/
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=0)


In [ ]:
# Converting the above pandas series to dataframe
train_df = pd.DataFrame({ 'label': y_train, 'img_path': x_train }).reset_index(drop=True)
val_df   = pd.DataFrame({ 'label': y_val, 'img_path': x_val }).reset_index(drop=True)
test_df  = pd.DataFrame({ 'label': y_test, 'img_path': x_test }).reset_index(drop=True)

# Printing the sets size
print(f'Training set size: {len(train_df)}')
print(f'Validation set size: {len(val_df)}')
print(f'Testing set size: {len(test_df)}')


In [ ]:
train_df.sample(5)


In [ ]:
# Looking at first 25 training examples

plt.figure(figsize=(15, 15))
for i, _img_path in enumerate(x_train[:25]):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.title('')
    img = load_img(_img_path)
    plt.imshow(img)


In [ ]:
# Global variables

IMG_HEIGHT = 150
IMG_WIDTH  = 150
CHANNELS   = 3

BATCH_SIZE = 128


In [ ]:
def process_images(list_of_images):
    x = []  # holds images
    y = []  # hold labels
    
    for image in list_of_images:
        x.append(
            cv2.resize(
                cv2.imread(image, cv2.IMREAD_COLOR),
                (IMG_HEIGHT, IMG_WIDTH),
                interpolation=cv2.INTER_CUBIC
            )
        )
        
        if 'dog' in image:
            y.append(1)
        if 'cat' in image:
            y.append(0)
    
    return x, y


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=10, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range = 0.1,
)

val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='img_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='img_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    # shuffle=True,
    class_mode='binary'
)


In [ ]:
def get_VGG16_model(input_shape):
    model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(model.output)
    Dropout(0.5)(x)
    Dense(4096, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(model.input, output)
    
    return model


def get_InceptionV3_model(input_shape):
    model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(model.output)
    Dropout(0.5)(x)
    Dense(4096, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(model.input, output)
    
    return model


model = get_InceptionV3_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))
model.summary()
print()


In [ ]:
# Building the CustomCallback for getting more info on model's performance
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        precision = logs['precision']
        recall = logs['recall']
        f1_score = (2 * (precision * recall)) / (precision + recall)
        
        loss = logs['loss']
        accuracy = logs['accuracy']
        auc_roc = logs['auc_roc']
        
        # Validation
        val_loss = logs['val_loss']
        val_accuracy = logs['val_accuracy']
        val_auc_roc = logs['val_auc_roc']
        val_precision = logs['val_precision']
        val_recall = logs['val_recall']
        val_f1_score = (2 * (val_precision * val_recall)) / (val_precision + val_recall)
        
        info = {
            'loss': round(loss, 5),
            'accuracy': round(accuracy, 4),
            'auc_roc': round(auc_roc, 4),
            'precision': round(precision, 4),
            'recall': round(recall, 4),
            'f1_score': round(f1_score, 4),
            'val_loss': round(val_loss, 5),
            'val_accuracy': round(val_accuracy, 4),
            'val_auc_roc': round(val_auc_roc, 4),
            'val_precision': round(val_precision, 4),
            'val_recall': round(val_recall, 4),
            'val_f1_score': round(val_f1_score, 4),
        }
        
        print(f'\n{json.dumps(info, indent=2)}')
        print()

        

callbacks = [
    ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=2, min_lr=0.000001, verbose=1),
    CustomCallback()
]


In [ ]:
num_epoch = 1
optimizer = RMSprop(learning_rate=0.0001)
loss = 'binary_crossentropy'

metrics = [
    'accuracy', 
    AUC(curve='ROC', name='auc_roc'), 
    Precision(name='precision'), 
    Recall(name='recall')
]


In [ ]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)


In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
    batch_size=BATCH_SIZE, 
    validation_data=val_generator, 
    epochs=num_epoch,
    callbacks=callbacks
)


In [ ]:
# Accuracy

plt.plot(history.history['accuracy'][1:], label='train acc')
plt.plot(history.history['val_accuracy'][1:], label='validation acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')


In [ ]:
# Loss

plt.plot(history.history['loss'][1:], label='train loss')
plt.plot(history.history['val_loss'][1:], label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')


In [ ]:
top_layer = model.layers[1]
plt.imshow(top_layer.get_weights()[0][:, :, :, 0].squeeze())


In [ ]:
def plot_filters_for_conv_layer(model, layer_index, num_columns=5, cmap='binary', how_many='all'):
    layer = model.layers[layer_index]
    filter_weights = layer.get_weights()[0]
    
    num_filters = layer.filters if how_many == 'all' else how_many
    num_rows = (num_filters // num_columns) + (num_filters % num_columns)
    # example:
    # num_rows = (96 // 5) + (96 % 5) == 20 (to plot all the filters)
    
    f, axs = plt.subplots(num_rows, num_columns, figsize=(20, 5 * num_rows))
    row_count = 0  # to plot num_columns figs in an individual row
    
    if not isinstance(axs, np.ndarray):
        # When num_cloumns == how_many
        axs = np.array(axs)  # to make axs iterable
        # list can also be inplace np.array but since plt.subplots axs output is of type np.ndarray I kept 
        
    for idx, row_ax in enumerate(axs):
        # plotting filters in a row
        for i, ax in enumerate(row_ax):
            if row_count + i >= num_filters:
                break
                
            if len(filter_weights.shape) == 4:
                if filter_weights.shape[2] == 1:
                    # For plotting filters whose weight shape is == (kernel_size_x, kernel_size_y, 1, #filters)
                    # example: (11, 11, 1, 96)
                    ax.imshow(filter_weights[:, :, :, row_count + i].squeeze(), cmap=cmap)
                else:
                    # For plotting filters whose weight shape is == (kernel_size_x, kernel_size_y, num > 1, #filters)
                    # example: (5, 5, 96, 256)
                    # because if ax.imshow(filter_weights[:, :, :, row_count + i].squeeze(), cmap=cmap)
                    # is used then we'll have array of (5, 5, 96) which is invalid image data for plotting 2D image
                    # (in above case where `filter_weights.shape[2] == 1` there we'll end up with (11, 11, 1) which
                    # after applying the `squeeze` function will be (11, 11) which is valid image data) so in 
                    # that case we'll just plot (5, 5) plot in the first 3D array i.e. (5, 5, 0, row_count + i) 
                    # => this is what we'll plot. To plot (5, 5, row_count + i, 0) just change indexing from
                    # [:, :, 0, row_count + i] to [:, :, row_count + i, 0]
                    ax.imshow(filter_weights[:, :, 0, row_count + i].squeeze(), cmap=cmap)
                    
                # For generalization this can be used, but to understand why 0 need to be used,
                # using the above way
                # ax.imshow(filter_weights[:, :, 0, row_count + i].squeeze(), cmap=cmap)
            else:
                break
                            
        # increasing row_count by num_columns
        row_count += num_columns 


In [ ]:
conv_layers_idxs = [idx for idx in range(len(model.layers)) if 'conv' in model.layers[idx].name]
print(len(conv_layers_idxs))

' | '.join([str(idx) for idx in conv_layers_idxs])


In [ ]:
plot_filters_for_conv_layer(
    model, 
    conv_layers_idxs[0], 
    cmap=sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True), 
    how_many=20
)  


In [ ]:
plot_filters_for_conv_layer(
    model, 
    conv_layers_idxs[1], 
    num_columns=5,
    how_many=10
)  


In [ ]:
def plot_feature_maps_for_single_conv_layer(model, layer_id, input_img, num_columns=10, cmap='binary'):
    ref_model = Model(inputs=model.inputs, outputs=model.layers[layer_id].output)
    feature_map = ref_model.predict(input_img)
    
    num_filters = feature_map[0].shape[2]
    num_rows = (num_filters // num_columns) + (num_filters % num_columns)

    fig = plt.figure(figsize=(16, 2 * num_rows))
    ix = 1
    for _ in range(num_rows):
        for _ in range(num_columns):
            if ix == num_filters:
                break
        
            # specify subplot and turn of axis
            ax = plt.subplot(num_rows, num_columns, ix)
            ax.set_xticks([])
            ax.set_yticks([])
        
            # plot filter channel in grayscale
            plt.imshow(feature_map[0, :, :, ix-1], cmap=cmap)
            ix += 1
            
    # show the figure
    plt.show()


In [ ]:
visualize_feature_maps_for = 0


In [ ]:
# Doing as `x_train` pd.Series index are not uniform, so resetting the index
img_path = x_train.reset_index(drop=True)[visualize_feature_maps_for]


In [ ]:
plt.imshow(load_img(img_path))


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[0], 
    process_images([img_path])[0][0][np.newaxis, ...], 
    num_columns=8,
)


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[2], 
    process_images([img_path])[0][0][np.newaxis, ...],
    cmap=sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True),
    num_columns=8,
)


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[len(conv_layers_idxs) // 2], 
    process_images([img_path])[0][0][np.newaxis, ...],
    cmap=sns.cubehelix_palette(as_cmap=True),
    num_columns=8,
)


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[-1], 
    process_images([img_path])[0][0][np.newaxis, ...],
    cmap=sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True),
    num_columns=8,
)


In [ ]:
visualize_feature_maps_for = 1

# Doing as `x_train` pd.Series index are not uniform, so resetting the index
img_path = x_train.reset_index(drop=True)[visualize_feature_maps_for]

plt.imshow(load_img(img_path))


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[0], 
    process_images([img_path])[0][0][np.newaxis, ...],
    cmap=sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True),
    num_columns=8,
)


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[2], 
    process_images([img_path])[0][0][np.newaxis, ...],
    cmap=sns.cubehelix_palette(as_cmap=True),
    num_columns=8,
)


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[len(conv_layers_idxs) // 2], 
    process_images([img_path])[0][0][np.newaxis, ...],
    num_columns=8,
)


In [ ]:
plot_feature_maps_for_single_conv_layer(
    model, 
    conv_layers_idxs[-1], 
    process_images([img_path])[0][0][np.newaxis, ...],
    cmap=sns.cubehelix_palette(as_cmap=True),
    num_columns=8,
)


In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix'):
    plt.figure(figsize=(4, 4))
    
    plt.imshow(cm, interpolation='nearest', cmap=sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True))
    plt.title(title)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
# Cat = 0, Dog = 1
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    x_col='img_path',
    y_col='label',    
    batch_size=BATCH_SIZE, 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)


In [ ]:
# Evaluating labelled test data
test_results = model.evaluate(train_generator, steps=x_test.shape[0] // BATCH_SIZE)

print()

print(f'Test Loss: {test_results[0]}')
print(f'Test Accuracy: {test_results[1]}')
print(f'Test ACU: {test_results[2]}')
print(f'Test Precision: {test_results[3]}')
print(f'Test Recall: {test_results[4]}')

f1_score_result = 2 * (test_results[3] * test_results[4]) / (test_results[3] + test_results[4])
print(f'Test F1 Score: {f1_score_result}')


In [ ]:
# Cat = 0, Dog = 1
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    x_col='img_path',
    y_col='label',    
    batch_size=BATCH_SIZE, 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

predictions = model.predict(test_generator, verbose=1)
predictions = predictions.flatten()

results = []
for i in predictions:
    if i >= 0.5:
        results.append(1)
    else:
        results.append(0)

plt.figure(figsize=(15, 15))
for i, _img_path in enumerate(test_generator.filenames[:25]):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    if results[i] == 0:
        plt.xlabel(f'Prediction: Cat')
    else:
        plt.xlabel(f'Prediction: Dog')
    img = load_img(_img_path)
    plt.imshow(img)
    
# compute the confusion matrix
confusion_mtx = confusion_matrix(test_generator.labels, results) 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=range(2)) 


In [ ]:
# Getting test data ready for prediction
test_imgs = [sample for sample in test_df.img_path]
x_test, y_test = process_images(test_imgs)
print(f'x_test length: {len(x_test)}, y_test length: {len(y_test)}')

x = np.asarray(x_test)

del x_test
gc.collect()

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(x, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
predictions = model.predict(test_generator, verbose=1)
predictions = predictions.flatten()

results = []
for i in predictions:
    if i >= 0.5:
        results.append(1)
    else:
        results.append(0)

plt.figure(figsize=(15, 15))
for i, _img_path in enumerate(test_imgs[:25]):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    if results[i] == 0:
        plt.xlabel(f'Prediction: Cat')
    else:
        plt.xlabel(f'Prediction: Dog')
    img = load_img(_img_path)
    plt.imshow(img)


In [ ]:
# compute the confusion matrix
labels_list = test_df.label.astype('int')
confusion_mtx = confusion_matrix(labels_list, results) 

# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=range(2)) 


In [ ]:
TEST_DIR = '../data/test/'
test_imgs = [TEST_DIR + i for i in os.listdir(TEST_DIR)]


In [ ]:
# Getting test data ready for prediction
x_test, y_test = process_images(test_imgs)
print(f'x_test length: {len(x_test)}, y_test length: {len(y_test)}')

x = np.asarray(x_test)

del x_test
gc.collect()

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(x, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
predictions = model.predict(test_generator, verbose=1)
predictions = predictions.flatten()

results = []
for i in predictions:
    if i >= 0.5:
        results.append(1)
    else:
        results.append(0)


In [ ]:
print(predictions[:10])
print()
print(results[:10])


In [ ]:
# Cat = 0, Dog = 1

plt.figure(figsize=(15, 15))
for i, _img_path in enumerate(test_imgs[:25]):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    if results[i] == 0:
        plt.xlabel(f'Prediction: Cat')
    else:
        plt.xlabel(f'Prediction: Dog')
    # plt.xlabel(f'Prediction: {predictions[i]}')
    img = load_img(_img_path)
    plt.imshow(img)


In [ ]:
submission = pd.read_csv('../input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
submission.head()


In [ ]:
for i in submission.index:
    # submission['label'].iloc[i] = results[i]
    submission['label'].iloc[i] = predictions[i]


In [ ]:
submission.to_csv('sample_submission.csv', index=False)
submission.head()


In [ ]:
model.save('model')
